# Домашнє завдання до теми "Розширені методи обробки даних у Pandas"

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [1]:
import pandas as pd
import sys

sys.path.insert(0, '../../utils')
from google_drive_utils import get_download_url


In [2]:
data_url = 'https://drive.google.com/file/d/1BxCDGcoPO08q65NBLCSsJOeIGet-b4dR/view?usp=sharing'
download_url = get_download_url(data_url)

df = pd.read_csv(download_url)

1. Відсортуйте дані за спаданням ціни товара ('Unit price') і виведіть 5 перших рядочків на екран.

In [5]:
df.sort_values(by='Unit price', ascending=False).head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
983,148-41-7930,C,Naypyitaw,Normal,Male,Health and beauty,99.96,7,34.986,734.706,1/23/2019,10:33,Cash,699.72,4.761905,34.986,6.1
122,219-22-9386,B,Mandalay,Member,Male,Sports and travel,99.96,9,44.982,944.622,3/9/2019,17:26,Credit card,899.64,4.761905,44.982,4.2
930,641-62-7288,B,Mandalay,Normal,Male,Home and lifestyle,99.92,6,29.976,629.496,3/24/2019,13:33,Ewallet,599.52,4.761905,29.976,7.1
494,437-53-3084,B,Mandalay,Normal,Male,Fashion accessories,99.89,2,9.989,209.769,2/26/2019,11:48,Ewallet,199.78,4.761905,9.989,7.1
283,667-92-0055,A,Yangon,Member,Male,Health and beauty,99.83,6,29.949,628.929,3/4/2019,15:02,Ewallet,598.98,4.761905,29.949,8.5


2. Знайдіть інвойс (рядок в даних) в категорії товарів Electronic accessories, в якому був куплений товар з найбільшою кількістю і найбільшою ціною. Виведіть цей рядок даних на екран.

In [15]:
df[df['Product line'] == 'Electronic accessories'].sort_values(by = ['Quantity', 'Unit price'], ascending=False).iloc[0]

Invoice ID                            704-48-3927
Branch                                          A
City                                       Yangon
Customer type                              Member
Gender                                       Male
Product line               Electronic accessories
Unit price                                  88.67
Quantity                                       10
Tax 5%                                     44.335
Total                                     931.035
Date                                    1/12/2019
Time                                        14:50
Payment                                   Ewallet
cogs                                        886.7
gross margin percentage                  4.761905
gross income                               44.335
Rating                                        7.3
Name: 105, dtype: object

3.1. Створіть нову колонку в даних `Rating Category`, яка визначає категорію рейтингу: високий, середній чи низький за наступним правилом:
- якщо рейтинг вищий або рівний 8 - то він високий (`High`)
- якщо рейтинг від 6 до 8 не включаючи - то він середній (`Medium`)
- інакше - низький (`Low`).

In [21]:
def rating_cat_mapping(val):
  if val >= 8:
    return 'High'
  elif val < 8 and val >= 6:
    return 'Medium'
  else:
    return 'Low' 
  
df['Rating Category'] = df['Rating'].apply(rating_cat_mapping)

df[df.Rating <= 6][['Rating','Rating Category']].sort_values(by = 'Rating')

,Rating,Rating Category
328,4.0,Low
848,4.0,Low
72,4.0,Low
379,4.0,Low
618,4.0,Low
...,...,...
735,6.0,Medium
904,6.0,Medium
636,6.0,Medium
642,6.0,Medium


3.2. Виведіть, який відсоток даних становить кожне зі значень категорій рейтингу.

In [22]:
df['Rating Category'].value_counts(normalize=True)

Rating Category
Medium    0.356
High      0.329
Low       0.315
Name: proportion, dtype: float64

4.1. Запустіть код нижче аби створити змінну `time`. Ця змінна містить час в форматі "година:хвилина". Напишіть функцію `get_hour`, яка буде зі змінної формату як `time` дістати лише години і конвертувати їх у тип `int`.

Запустіть написану функцію на змінній `time` та перевірте правильність роботи функції.

In [23]:
time = '11:28'

In [24]:
def get_hour(val):
  hour = val.split(':')[0]
  return int(hour)

get_hour(time)

11

4.2. З допомогою функції `get_hour` і метода `DataFrame.apply()` створіть нову колонку `Hour` в нашому наборі даних, яка буде містити години здійснення покупки, обчислені на основі часу здійснення покупки `Time`.

In [25]:
df['Hour'] = df['Time'].apply(get_hour)

4.3. Виведіть кількість значень за кожною годиною покупки і дайте відповідь "О яких годинах найбільше здійснюють покупок покупці і чому на Ваш погляд саме о цих годинах? Чи це слогічно, що найбільше покупок саме в ці години?".

In [30]:
df['Hour'].value_counts()

Hour
19    113
13    103
15    102
10    101
18     93
11     90
12     89
14     83
16     77
20     75
17     74
Name: count, dtype: int64

Найбільше покупки здійснюються о сьомій вечора, що є логічним, якщо припустити, що покупці здійснюють покупки після роботи. Покупки о першій годині можуть свідчити про людей, що виходять у магазин під час обідньої перерви. Покупки о третій годині дня можуть свідчити про людей, що закінчують працювати раніше, наприклад, вчителі у школі, а також школяри чи студенти, якщо поруч є навчальні заклади. Покупки о десятій, що є мабуть годиною відкриття магазину, скоріш за все свідчать про людей, що чекали відкриття магазину вранці, можливо для придбання товарів першої необхідності. Варто проаналізувати які саме групи товарів були придбані у цей час. 

5. Виведіть середнє значення чеку (`Total`) для покупок в містах, наявних в даних, за допомогою функції `groupby`.

In [31]:
df.groupby(by='City')['Total'].mean()

City
Mandalay     319.872506
Naypyitaw    337.099715
Yangon       312.354031
Name: Total, dtype: float64

6. Виведіть кількість покупок, середнє, мінімальне та максимальне значення для загальної суми чеку (Total) в містах, наявних в даних, за допомогою функції groupby.
Зробіть висновки з цих показників та дайте рекомендації для мережі супермаркетів.

In [35]:
df.groupby(by='City')['Total'].agg(['count','mean', 'min', 'max'])

,count,mean,min,max
City,,,,
Mandalay,332,319.872506,18.6375,1022.49
Naypyitaw,328,337.099715,10.6785,1042.65
Yangon,340,312.354031,12.6945,1039.29


В цілому прибутковість супермаркетів однакова. 

7. Ми внесли деякі зміни в наш набір даних, тож, давайте запишемо результат у файл. Запишіть оновлену версію даних у файл `supermarket_sales_enriched.csv`. Зверніть увагу, аби не записати нічого зайвого, наприклад, індекси рядків нам в файлі не потрібні, оскільки вони стандартні.

In [37]:
df.to_csv('../data/supermarket_sales_enriched.csv', index=False)